<a href="https://colab.research.google.com/github/cbadenes/curso-pln/blob/main/notebooks/proyecto_apoyo/02_RecuperacionInformacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#RECUPERADORES DISPERSOS

##TF-IDF

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Base de conocimiento
data = [
    "Resetear contraseña", "Encontrar número de serie", "Dispositivo no enciende"
]
vectorizer = TfidfVectorizer()
vector_matrix = vectorizer.fit_transform(data)

# Consulta del usuario
query = "olvidé mi contraseña"
#query = "olvidé mi clave"
query_vec = vectorizer.transform([query])
similarities = cosine_similarity(query_vec, vector_matrix).flatten()

# Mostrar resultados
ranking = similarities.argsort()[::-1]
for idx in ranking:
    print(f"Documento: {data[idx]} | Similitud: {similarities[idx]:.2f}")


Documento: Resetear contraseña | Similitud: 0.71
Documento: Dispositivo no enciende | Similitud: 0.00
Documento: Encontrar número de serie | Similitud: 0.00


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Crear DataFrame para mostrar vectores
df_docs = pd.DataFrame(vector_matrix.toarray(), columns=vectorizer.get_feature_names_out(), index=data)
df_query = pd.DataFrame(query_vec.toarray(), columns=vectorizer.get_feature_names_out(), index=["Consulta"])
df_total = pd.concat([df_docs, df_query])

# Mostrar los vectores
print("Vectores TF-IDF:")
print(df_total)


Vectores TF-IDF:
                           contraseña   de  dispositivo  enciende  encontrar  \
Resetear contraseña          0.707107  0.0      0.00000   0.00000        0.0   
Encontrar número de serie    0.000000  0.5      0.00000   0.00000        0.5   
Dispositivo no enciende      0.000000  0.0      0.57735   0.57735        0.0   
Consulta                     1.000000  0.0      0.00000   0.00000        0.0   

                                no  número  resetear  serie  
Resetear contraseña        0.00000     0.0  0.707107    0.0  
Encontrar número de serie  0.00000     0.5  0.000000    0.5  
Dispositivo no enciende    0.57735     0.0  0.000000    0.0  
Consulta                   0.00000     0.0  0.000000    0.0  


#RECUPERADORES DENSOS

##WORD2VEC

In [ ]:
!pip install numpy==1.26.4 scipy==1.13.1 gensim==4.3.3 spacy==3.7.5

In [ ]:
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Base de conocimiento
data = ["Resetear contraseña", "Encontrar número de serie", "Dispositivo no enciende"]
sentences = [frase.lower().split() for frase in data]

# Entrenar modelo Word2Vec
model = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=1, workers=2)

# Promediar vectores por frase
def sentence_vector(sentence, model):
    words = sentence.lower().split()
    vectors = [model.wv[word] for word in words if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

data_vectors = [sentence_vector(texto, model) for texto in data]

# Consulta
query = "olvidé mi clave"
query_vec = sentence_vector(query, model)

# Similaridades
similarities = cosine_similarity([query_vec], data_vectors).flatten()

# Ranking
ranking = similarities.argsort()[::-1]
for idx in ranking:
    print(f"Documento: {data[idx]} | Similitud: {similarities[idx]:.2f}")


Documento: Dispositivo no enciende | Similitud: 0.00
Documento: Encontrar número de serie | Similitud: 0.00
Documento: Resetear contraseña | Similitud: 0.00


##FastText

In [ ]:
from gensim.models import FastText
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Base de conocimiento
data = [
    "Resetear contraseña",
    "Encontrar número de serie",
    "Dispositivo no enciende"
]

# Preprocesamiento: convertir a listas de palabras
tokenized_data = [frase.lower().split() for frase in data]

# Entrenar modelo FastText desde cero
model = FastText(sentences=tokenized_data, vector_size=100, window=3, min_count=1, epochs=50)

# Función para vectorizar una frase (media de los vectores de las palabras)
def sentence_vector(sentence, model):
    words = sentence.lower().split()
    vectors = [model.wv[word] for word in words if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

# Vectorizar base de conocimiento
data_vectors = [sentence_vector(frase, model) for frase in data]

# Consulta
query = "olvidé mi clave"
query_vec = sentence_vector(query, model)

# Calcular similitud de coseno
similarities = cosine_similarity([query_vec], data_vectors).flatten()

# Mostrar ranking
ranking = similarities.argsort()[::-1]
for idx in ranking:
    print(f"Documento: {data[idx]} | Similitud: {similarities[idx]:.2f}")


Documento: Dispositivo no enciende | Similitud: 0.05
Documento: Encontrar número de serie | Similitud: -0.02
Documento: Resetear contraseña | Similitud: -0.10


In [ ]:
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Cargar modelo FastText preentrenado en español
#model = KeyedVectors.load_word2vec_format('cc.es.300.vec.gz', binary=False)

# Base de conocimiento
data = ["Resetear contraseña", "Encontrar número de serie", "Dispositivo no enciende"]

# Promediar vectores de cada frase
def sentence_vector(sentence, model):
    words = sentence.lower().split()
    vectors = [model[word] for word in words if word in model]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

data_vectors = [sentence_vector(texto, model) for texto in data]

# Consulta
query = "olvidé mi clave"
query_vec = sentence_vector(query, model)

# Similaridades
similarities = cosine_similarity([query_vec], data_vectors).flatten()

# Ranking
ranking = similarities.argsort()[::-1]
for idx in ranking:
    print(f"Documento: {data[idx]} | Similitud: {similarities[idx]:.2f}")


Documento: Dispositivo no enciende | Similitud: 0.34
Documento: Resetear contraseña | Similitud: 0.33
Documento: Encontrar número de serie | Similitud: 0.31


##TRANSFORMERS


###Modelo preentrenado: paraphrase-multilingual-MiniLM-L12-v2


In [ ]:
from sentence_transformers import SentenceTransformer, util

# Base de conocimiento
data = [
    "Resetear contraseña", "Encontrar número de serie", "Dispositivo no enciende"
]
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
data_embeddings = model.encode(data, convert_to_tensor=True)

# Consulta del usuario
#query = "olvidé mi contraseña"
query = "olvidé mi clave"
query_embedding = model.encode(query, convert_to_tensor=True)
similarities = util.pytorch_cos_sim(query_embedding, data_embeddings).squeeze()

# Mostrar resultados
ranking = similarities.argsort(descending=True)
for idx in ranking:
    print(f"Documento: {data[idx]} | Similitud: {similarities[idx]:.2f}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Documento: Resetear contraseña | Similitud: 0.55
Documento: Dispositivo no enciende | Similitud: 0.32
Documento: Encontrar número de serie | Similitud: 0.11


###Modelo distiluse-base-multilingual-cased-v1


In [1]:
from sentence_transformers import SentenceTransformer, util

# Base de conocimiento
data = [
    "Resetear contraseña", "Encontrar número de serie", "Dispositivo no enciende"
]
model = SentenceTransformer("distiluse-base-multilingual-cased-v1")
data_embeddings = model.encode(data, convert_to_tensor=True)

# Consulta del usuario
#query = "olvidé mi contraseña"
query = "olvidé mi clave"
query_embedding = model.encode(query, convert_to_tensor=True)
similarities = util.pytorch_cos_sim(query_embedding, data_embeddings).squeeze()

# Mostrar resultados
ranking = similarities.argsort(descending=True)
for idx in ranking:
    print(f"Documento: {data[idx]} | Similitud: {similarities[idx]:.2f}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.24k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

2_Dense/pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

Documento: Resetear contraseña | Similitud: 0.46
Documento: Encontrar número de serie | Similitud: 0.19
Documento: Dispositivo no enciende | Similitud: 0.15


#Ejemplo de comparación temporal

##Calculando embeddings

In [ ]:
!pip install -U datasets

In [5]:
from datasets import load_dataset
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util

# Cargar un subset del dataset de Wikipedia en español
dataset = load_dataset("wikimedia/wikipedia","20231101.es", split="train",trust_remote_code=True)
data = [item["text"] for item in dataset.select(range(20000)) if isinstance(item["text"], str)]

# Consulta del usuario
query = "¿Cuándo finalizó la Segunda Guerra Mundial?"

# **TF-IDF**
vectorizer = TfidfVectorizer()
start_time_tfidf = time.time()
vector_matrix = vectorizer.fit_transform(data)
query_vec = vectorizer.transform([query])
tfidf_similarities = cosine_similarity(query_vec, vector_matrix).flatten()
end_time_tfidf = time.time()

# Ordenar resultados TF-IDF
ranking_tfidf = tfidf_similarities.argsort()[::-1]
top_tfidf = [(data[idx], tfidf_similarities[idx], dataset['title'][idx]) for idx in ranking_tfidf[:5]]

# Embeddings
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")  # Modelo multilingüe para español
start_time_embeddings = time.time()
data_embeddings = model.encode(data, convert_to_tensor=True)
query_embedding = model.encode(query, convert_to_tensor=True)
embedding_similarities = util.pytorch_cos_sim(query_embedding, data_embeddings).squeeze().cpu().numpy()
end_time_embeddings = time.time()

# Ordenar resultados Embeddings
ranking_embeddings = embedding_similarities.argsort()[::-1]
top_embeddings = [(data[idx], embedding_similarities[idx],dataset['title'][idx]) for idx in ranking_embeddings[:5]]

# Mostrar resultados
print("=== Resultados TF-IDF ===")
print(f"Tiempo de cálculo: {end_time_tfidf - start_time_tfidf:.4f} segundos")
for idx, (doc, sim, tit) in enumerate(top_tfidf):
    print(f"\nDocumento {idx+1}:")
    print(f"Similitud: {sim:.4f}\n{tit}\n{doc[:300]}")

print("\n=== Resultados Embeddings ===")
print(f"Tiempo de cálculo: {end_time_embeddings - start_time_embeddings:.4f} segundos")
for idx, (doc, sim,tit) in enumerate(top_embeddings):
    print(f"\nDocumento {idx+1}:")
    print(f"Similitud: {sim:.4f}\n{tit}\n{doc[:300]}")



=== Resultados TF-IDF ===
Tiempo de cálculo: 36.9988 segundos

Documento 1:
Similitud: 0.2366
Guerra mundial
Guerra mundial es un término utilizado para referirse a una batalla que puede durar días, semanas, meses o hasta años y que puede involucrar directamente a varias naciones de distintos continentes.

Discrepancias en el término 
Pese a ser aceptadas las varias guerras mundiales, publicaciones como lo

Documento 2:
Similitud: 0.2112
Guerra
La guerra, estrictamente hablando, es aquel conflicto social en el que dos o más grupos humanos relativamente masivos —principalmente tribus, sociedades o naciones— se enfrentan de manera violenta, generalmente mediante el uso de armas de toda índole, a menudo con resultado de muerte —individual o c

Documento 3:
Similitud: 0.1803
Conferencia de París (1945)
La conferencia de París de 1945 (9 de noviembre al 21 de diciembre), fue una reunión celebrada por los representantes de los Estados aliados para decidir las reparaciones de guerra a imponer

##Cargando embeddings pre-calculados

Ahora guardo los modelos, embeddings y vectores para agilizar el proceso.

In [6]:
#Guardo los embeddings y vectores ya generados anteriormente
import pickle

with open('data_embeddings.pkl', 'wb') as f:
    pickle.dump(data_embeddings, f)

with open('vector_matrix.pkl', 'wb') as f:
    pickle.dump(vector_matrix, f)

with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

#guardo el modelo
model.save('model')


In [7]:
#Guardo los datos (data y dataset)
import pickle

with open('data.pkl', 'wb') as f:
    pickle.dump(data, f)

with open('dataset.pkl', 'wb') as f:
    pickle.dump(dataset, f)

In [8]:
from datasets import load_dataset
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util

# Cargar los datos
with open('data.pkl', 'rb') as f:
    data = pickle.load(f)
with open('dataset.pkl', 'rb') as f:
    dataset = pickle.load(f)

# Consulta del usuario
query = "¿Cuándo finalizó la Segunda Guerra Mundial?"

# **TF-IDF**
# Cargo vectorizador y vectores
with open('vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)
start_time_tfidf = time.time()
with open('vector_matrix.pkl', 'rb') as f:
    vector_matrix = pickle.load(f)
query_vec = vectorizer.transform([query])
tfidf_similarities = cosine_similarity(query_vec, vector_matrix).flatten()
end_time_tfidf = time.time()

# Ordenar resultados TF-IDF
ranking_tfidf = tfidf_similarities.argsort()[::-1]
top_tfidf = [(data[idx], tfidf_similarities[idx], dataset['title'][idx]) for idx in ranking_tfidf[:5]]

# **Embeddings**
#cargo modelo y embeddings
model = SentenceTransformer('model')
start_time_embeddings = time.time()
with open('data_embeddings.pkl', 'rb') as f:
    data_embeddings = pickle.load(f)
query_embedding = model.encode(query, convert_to_tensor=True)
embedding_similarities = util.pytorch_cos_sim(query_embedding, data_embeddings).squeeze().cpu().numpy()
end_time_embeddings = time.time()

# Ordenar resultados Embeddings
ranking_embeddings = embedding_similarities.argsort()[::-1]
top_embeddings = [(data[idx], embedding_similarities[idx],dataset['title'][idx]) for idx in ranking_embeddings[:5]]

# Mostrar resultados
print("=== Resultados TF-IDF ===")
print(f"Tiempo de cálculo: {end_time_tfidf - start_time_tfidf:.4f} segundos")
for idx, (doc, sim, tit) in enumerate(top_tfidf):
    print(f"\nDocumento {idx+1}:")
    print(f"Similitud: {sim:.4f}\n{tit}\n{doc[:300]}")

print("\n=== Resultados Embeddings ===")
print(f"Tiempo de cálculo: {end_time_embeddings - start_time_embeddings:.4f} segundos")
for idx, (doc, sim,tit) in enumerate(top_embeddings):
    print(f"\nDocumento {idx+1}:")
    print(f"Similitud: {sim:.4f}\n{tit}\n{doc[:300]}")

=== Resultados TF-IDF ===
Tiempo de cálculo: 1.9000 segundos

Documento 1:
Similitud: 0.2366
Guerra mundial
Guerra mundial es un término utilizado para referirse a una batalla que puede durar días, semanas, meses o hasta años y que puede involucrar directamente a varias naciones de distintos continentes.

Discrepancias en el término 
Pese a ser aceptadas las varias guerras mundiales, publicaciones como lo

Documento 2:
Similitud: 0.2112
Guerra
La guerra, estrictamente hablando, es aquel conflicto social en el que dos o más grupos humanos relativamente masivos —principalmente tribus, sociedades o naciones— se enfrentan de manera violenta, generalmente mediante el uso de armas de toda índole, a menudo con resultado de muerte —individual o c

Documento 3:
Similitud: 0.1803
Conferencia de París (1945)
La conferencia de París de 1945 (9 de noviembre al 21 de diciembre), fue una reunión celebrada por los representantes de los Estados aliados para decidir las reparaciones de guerra a imponer 